In [ ]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

### Loading the dataset

In [25]:
# Loading dataset
dataset_dir_train = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train"
dataset_dir_test = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test"

train_transform = transforms.Compose([transforms.Resize((300, 300)),
                                 transforms.RandomRotation(15),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
test_transform = transforms.Compose([transforms.Resize((300, 300)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir_test, transform = test_transform)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32, shuffle=False, num_workers = 2)

### Lables

In [4]:
# print the lables
names = list(pd.read_csv("../input/stanford-car-dataset-by-classes-folder/names.csv", names = ['names'])['names'])
names.sort()
print(names)

['AM General Hummer SUV 2000', 'Acura Integra Type R 2001', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi A5 Coupe 2012', 'Audi R8 Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi S4 Sedan 2007', 'Audi S4 Sedan 2012', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S6 Sedan 2011', 'Audi TT Hatchback 2011', 'Audi TT RS Coupe 2012', 'Audi TTS Coupe 2012', 'Audi V8 Sedan 1994', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW Z4 Convertible 2012

### Function to train model

In [16]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = lossFun(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
    print('Training Finished')
    return model, losses, accuracies

### Function to evaluate model

In [17]:
# function to evaluate a model
def EvaluateModel(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            # assigning to GPU
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * (correct / total)
    print('The accuracy on the test dataset: %d %%' % (
        test_acc))
    return test_acc

### Downloading pretrained models

In [5]:
# downloading the pretrained models
model_dense161 = models.densenet161(pretrained=True)
model_res152 = models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

### Densenet161 + SGD + CrossEntropyLoss

In [12]:
num_ftrs = model_dense161.classifier.in_features
model_dense161.classifier = nn.Linear(num_ftrs, 196)
model_dense161 = model_dense161.to(device)


lossFun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_dense161.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

In [14]:
model_dense161, training_losses, training_accs = TrainModel(model_dense161, lossFun, optimizer, lrscheduler, n_epochs=10)

1963.0 8144
Epoch 1, Time Used: 226 s, loss: 0.1098, acc: 24.1036
5592.0 8144
Epoch 2, Time Used: 213 s, loss: 0.0367, acc: 68.6640
6918.0 8144
Epoch 3, Time Used: 213 s, loss: 0.0178, acc: 84.9460
7555.0 8144
Epoch 4, Time Used: 214 s, loss: 0.0091, acc: 92.7677
7775.0 8144
Epoch 5, Time Used: 211 s, loss: 0.0058, acc: 95.4691
7883.0 8144
Epoch 6, Time Used: 211 s, loss: 0.0039, acc: 96.7952
8059.0 8144
Epoch 7, Time Used: 211 s, loss: 0.0019, acc: 98.9563
8090.0 8144
Epoch 8, Time Used: 212 s, loss: 0.0013, acc: 99.3369
8094.0 8144
Epoch 9, Time Used: 211 s, loss: 0.0012, acc: 99.3861
8110.0 8144
Epoch 10, Time Used: 214 s, loss: 0.0010, acc: 99.5825
Training Finished


In [23]:
EvaluateModel(model_dense161)

The accuracy on the test dataset: 90 %


90.07586121129214

### Resnet152 + SGD + CrossEntropyLoss

In [28]:
num_ftrs = model_res152.fc.in_features
model_res152.fc = nn.Linear(num_ftrs, 196)
model_res152 = model_res152.to(device)

lossFun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_res152.parameters(), lr=0.01, momentum=0.9)

lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
model_res152, training_losses, training_accs = TrainModel(model_res152, lossFun, optimizer, lrscheduler, n_epochs=10)

1500.0 8144
Epoch 1, Time Used: 215 s, loss: 0.1166, acc: 18.4185
4718.0 8144
Epoch 2, Time Used: 215 s, loss: 0.0478, acc: 57.9322
6131.0 8144
Epoch 3, Time Used: 214 s, loss: 0.0271, acc: 75.2824
6734.0 8144
Epoch 4, Time Used: 216 s, loss: 0.0181, acc: 82.6866
7257.0 8144
Epoch 5, Time Used: 215 s, loss: 0.0118, acc: 89.1085
7490.0 8144
Epoch 6, Time Used: 215 s, loss: 0.0083, acc: 91.9695
7899.0 8144
Epoch 7, Time Used: 215 s, loss: 0.0038, acc: 96.9917
8004.0 8144
Epoch 8, Time Used: 214 s, loss: 0.0024, acc: 98.2809
8045.0 8144
Epoch 9, Time Used: 214 s, loss: 0.0020, acc: 98.7844
8049.0 8144
Epoch 10, Time Used: 214 s, loss: 0.0017, acc: 98.8335
Training Finished


In [30]:
model_res152.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
EvaluateModel(model_res152)

The accuracy on the test dataset: 88 %


88.57107324959583

### Densenet161 + Adam + CrossEntropyLoss

In [15]:
num_ftrs = model_dense161.classifier.in_features

# replace the last fc layer with an untrained one (requires grad by default)
model_dense161.classifier = nn.Linear(num_ftrs, 196)
model_dense161 = model_dense161.to(device)


lossFun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_dense161.parameters(), lr=0.01, betas=(0.9, 0.999))
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

In [16]:
model_dense161, training_losses, training_accs = TrainModel(model_dense161, lossFun, optimizer, lrscheduler, n_epochs=10)

47.0 8144
Epoch 1, Time Used: 198 s, loss: 0.1783, acc: 0.5771
67.0 8144
Epoch 2, Time Used: 198 s, loss: 0.1654, acc: 0.8227
99.0 8144
Epoch 3, Time Used: 197 s, loss: 0.1632, acc: 1.2156
101.0 8144
Epoch 4, Time Used: 197 s, loss: 0.1620, acc: 1.2402
117.0 8144
Epoch 5, Time Used: 197 s, loss: 0.1605, acc: 1.4366
129.0 8144
Epoch 6, Time Used: 199 s, loss: 0.1593, acc: 1.5840
120.0 8144
Epoch 7, Time Used: 198 s, loss: 0.1585, acc: 1.4735
182.0 8144
Epoch 8, Time Used: 198 s, loss: 0.1563, acc: 2.2348
178.0 8144
Epoch 9, Time Used: 197 s, loss: 0.1556, acc: 2.1857
187.0 8144
Epoch 10, Time Used: 197 s, loss: 0.1553, acc: 2.2962
Training Finished


### Resnet152 + Adam + CrossEntropyLoss

In [6]:
num_ftrs = model_res152.fc.in_features
model_res152.fc = nn.Linear(num_ftrs, 196)
model_res152 = model_res152.to(device)

lossFun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_dense161.parameters(), lr=0.01, betas=(0.9, 0.999))
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
model_res152, training_losses, training_accs = TrainModel(model_res152, lossFun, optimizer, lrscheduler, n_epochs=10)

45.0 8144
Epoch 1, Time Used: 200 s, loss: 0.1671, acc: 0.5526
45.0 8144
Epoch 2, Time Used: 193 s, loss: 0.1671, acc: 0.5526
40.0 8144
Epoch 3, Time Used: 192 s, loss: 0.1671, acc: 0.4912
52.0 8144
Epoch 4, Time Used: 192 s, loss: 0.1671, acc: 0.6385
48.0 8144
Epoch 5, Time Used: 192 s, loss: 0.1671, acc: 0.5894
45.0 8144
Epoch 6, Time Used: 192 s, loss: 0.1671, acc: 0.5526
39.0 8144
Epoch 7, Time Used: 192 s, loss: 0.1671, acc: 0.4789
45.0 8144
Epoch 8, Time Used: 192 s, loss: 0.1671, acc: 0.5526
41.0 8144
Epoch 9, Time Used: 192 s, loss: 0.1670, acc: 0.5034
53.0 8144
Epoch 10, Time Used: 192 s, loss: 0.1671, acc: 0.6508
Training Finished
